In [6]:
from seirsplus.models import *
from seirsplus.networks import *
from seirsplus.sim_loops import *
from seirsplus.utilities import *
import networkx as nx
import time
import matplotlib.pyplot as plt
import pickle
import numpy as np
from seirsplus.utilities import load_model, save_model
from seirsplus.networks import prune_graph_per_age

In [2]:
G_baseline = networkx.read_gpickle("G_baseline.gpickle")
G_less_connections_1 = networkx.read_gpickle("G_less_connections_1.gpickle")
G_less_connections_2 = networkx.read_gpickle("G_less_connections_2.gpickle")
G_less_connections_3 = networkx.read_gpickle("G_less_connections_3.gpickle")
G_less_connections_4 = networkx.read_gpickle("G_less_connections_4.gpickle")
G_less_connections_5 = networkx.read_gpickle("G_less_connections_5.gpickle")
G_less_connections_6 = networkx.read_gpickle("G_less_connections_6.gpickle")
G_less_connections_7 = networkx.read_gpickle("G_less_connections_7.gpickle")

G_quarantine = networkx.read_gpickle("G_quarantine.gpickle")
households = pickle.load(open("households.p", "rb"))
individual_ageGroups = pickle.load( open("individual_ageGroups.p", "rb"))

In [3]:
individual_ages_indexes = [np.where(np.array(individual_ageGroups) == a)[0] for a in np.unique(np.array(individual_ageGroups))]
num_prune_per_age = np.array([10, 5, 10, 10, 5, 4, 20, 10, 2])

In [ ]:
self.S = 1
self.E = 2
self.I_pre = 3
self.I_sym = 4
self.I_asym = 5
self.H = 6
self.R = 7
self.F = 8
self.Q_S = 11
self.Q_E = 12
self.Q_pre = 13
self.Q_sym = 14
self.Q_asym = 15
self.Q_R = 17

In [ ]:

indexes = []
for group in self.nodeGroupData.keys():
    node_indexes = self.nodeGroupData[group]['nodes']
    current_state = self.X[node_indexes]
    s_indexes = numpy.where(current_state == self.S)[0]
    n_vaccinate = numpy.min(s_indexes.size, num_vaccinations_per_age[group])
    indexes_to_vaccinate = numpy.random.choice(s_indexes, size=n_vaccinate)
    indexes.extend(indexes_to_vaccinate)

In [12]:
N = 20000
model = load_model("model_second_wave_2021_04_10")

In [45]:
np.array(individual_ageGroups)[np.where(model.X == 4)[0]]

array(['0-9', '0-9', '0-9', '0-9', '0-9', '0-9', '0-9', '0-9', '0-9',
       '0-9', '0-9', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '30-39', '30-39', '30-39', '30-39', '30-39', '30-39',
       '20-29', '30-39', '30-39', '30-39', '30-39', '20-29', '30-39',
       '30-39', '30-39', '20-29', '30-39', '20-29', '20-29', '20-29',
       '20-29', '40-49', '40-49', '40-49', '40-49', '40-49', '40-49',
       '50-59', '50-59', '50-59', '50-59', '50-59', '50-59', '80+',
       '70-79', '70-79', '60-69', '60-69', '60-69', '80+', '60-69',
       '60-69', '60-69', '60-69', '70-79'], dtype='<U5')

In [29]:
node_ids = {}
for a in np.unique(individual_ageGroups):
    node_ids[a] = [i for i in range(len(individual_ageGroups)) if individual_ageGroups[i] == a]

In [34]:
p = [print(f"num {k} : {len(v)}") for k,v in node_ids.items()]

num 0-9 : 3759
num 10-19 : 3562
num 20-29 : 2825
num 30-39 : 2548
num 40-49 : 2514
num 50-59 : 1636
num 60-69 : 1532
num 70-79 : 1033
num 80+ : 591


In [ ]:
for a, n_ids in node_ids.items():
    nx.draw(G_baseline, )